In [ ]:
# IMPORTAR BIBLIOTECAS
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import yfinance as yf
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# CARREGAMENTO E PREPARAÇÃO DOS DADOS
ticker = "BOVA11.SA"
data = yf.download(ticker, start="2012-01-01", end="2024-12-31", auto_adjust=False)
data.columns = data.columns.get_level_values(0)

# Parâmetros
p = 5  # Período para cálculo de retorno
limite_superior = 0.015
limite_inferior = -0.015

# Calcular retornos
data["Return"] = data["Adj Close"].pct_change(p)
data["Target"] = data["Return"].shift(-p)

# Definir variáveis dos anos de treino e teste
year_train = "2022"
year_test = "2023"

In [ ]:
# TREINAMENTO DA CADEIA DE MARKOV
data_train = data.loc[f"{year_train}-01-01":f"{year_train}-12-31"].copy()

def classificar_retornos(x):
    if pd.isna(x):
        return None
    elif x > limite_superior:
        return "Aumento"
    elif x < limite_inferior:
        return "Diminuição"
    else:
        return "Estável"

# Classificar apenas dados de treinamento
data_train["Class"] = data_train["Return"].apply(classificar_retornos)
data_train = data_train.dropna(subset=["Class"])

# Calcular matriz de transição APENAS com dados de treinamento
estados = ["Aumento", "Diminuição", "Estável"]
transicao = {estado: {e: 0 for e in estados} for estado in estados}

for i in range(1, len(data_train)):
    estado_atual = data_train["Class"].iloc[i-1]
    prox_estado = data_train["Class"].iloc[i]
    transicao[estado_atual][prox_estado] += 1

# Calcular probabilidades
for estado_atual, transicoes in transicao.items():
    total = sum(transicoes.values())
    if total > 0:
        for estado in estados:
            transicao[estado_atual][estado] /= total

In [ ]:
# APLICAÇÃO DA ESTRATÉGIA (TESTE)
data_test = data.loc[f"{year_test}-01-01":f"{year_test}-12-31"].copy()
data_test["Class"] = data_test["Return"].apply(classificar_retornos)

threshold=0.6
signals = []
positions = []

for i in range(len(data_test)):
    if pd.isna(data_test["Class"].iloc[i]):
        signals.append("NEUTRO")
        positions.append(0)
        continue
        
    state = data_test["Class"].iloc[i]
    
    # Probabilidades de transição do estado atual para o estado futuro
    prob_aumento = transicao.get(state, {}).get("Aumento", 0)
    prob_diminuicao = transicao.get(state, {}).get("Diminuição", 0)
    
    # Lógica de trading simplificada
    if prob_aumento > threshold:
        signals.append("COMPRA")
        positions.append(1)
    elif prob_diminuicao > threshold:
        signals.append("VENDA")
        positions.append(-1)
    else:
        signals.append("NEUTRO")
        positions.append(0)

# Aplicar estratégia
data_test["Signal"] = signals 
data_test["Position"] = positions

# Calcular retornos da estratégia
data_test["Strategy_Return"] = data_test["Position"] * data_test["Target"]
data_test["Cumulative_Strategy"] = (1 + data_test["Strategy_Return"].fillna(0)).cumprod()

In [ ]:
# MÉTRICAS DE PERFORMANCE
clean_returns = data_test["Strategy_Return"].fillna(0)

total_return = (1 + clean_returns).prod() - 1
volatility = clean_returns.std() * np.sqrt(252)
sharpe = (clean_returns.mean() * 252) / (volatility) if volatility != 0 else 0

# Drawdown
cum_returns = (1 + clean_returns).cumprod()
running_max = cum_returns.expanding().max()
drawdown = (cum_returns - running_max) / running_max
max_drawdown = drawdown.min()

In [ ]:
# VISUALIZAÇÃO
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Performance da Estratégia', 'Sinais de Trading',
                   'Distribuição de Probabilidades', 'Drawdown'),
    vertical_spacing=0.12
)

fig.add_trace(
    go.Scatter(x=data_test.index, y=data_test["Cumulative_Strategy"],
              name="Markov Strategy", line=dict(color="red", width=2)),
    row=1, col=1
)

colors = {'COMPRA': 'green', 'VENDA': 'red', 'NEUTRO': 'gray'}
for signal in colors.keys():
    signal_data = data_test[data_test["Signal"] == signal]
    if len(signal_data) > 0:
        fig.add_trace(
            go.Scatter(x=signal_data.index, y=signal_data["Adj Close"],
                      mode="markers", name=signal, 
                      marker=dict(color=colors[signal], size=6)),
            row=1, col=2
        )

state_counts = data_test["Class"].value_counts()
fig.add_trace(
    go.Bar(x=state_counts.index, y=state_counts.values, 
           name="Distribuição Estados", marker_color='lightblue'),
    row=2, col=1
)

cum_strategy = data_test["Cumulative_Strategy"]
running_max = cum_strategy.expanding().max()
drawdown = (cum_strategy - running_max) / running_max

fig.add_trace(
    go.Scatter(x=data_test.index, y=drawdown, name="Drawdown",
              fill='tozeroy', line=dict(color="red")),
    row=2, col=2
)

fig.update_layout(
    height=800,
    title_text=f"Sistema de Trading com Cadeia de Markov - {ticker} ({year_test})",
    showlegend=True
)

fig.show()

In [ ]:
# RELATÓRIO FINAL
print("\n" + "="*50)
print("RELATÓRIO - MARKOV TRADING SYSTEM")
print("="*50)
print(f"Ativo: {ticker}")
print(f"Treinamento: {year_train}")
print(f"Teste: {year_test}")
print("-"*50)

print("\nMATRIZ DE TRANSIÇÃO (Treinamento):")
for estado, transicoes in transicao.items():
    print(f"{estado}:")
    for prox_estado, prob in transicoes.items():
        print(f"  → {prox_estado}: {prob:.3f}")

print(f"\nPERFORMANCE DA ESTRATÉGIA:")
print(f"Retorno Total: {total_return:.2%}")
print(f"Volatilidade: {volatility:.2%}")
print(f"Sharpe Ratio: {sharpe:.3f}")
print(f"Max Drawdown: {max_drawdown:.2%}")
print(f"Número de Trades: {len(clean_returns[clean_returns != 0])}")

print(f"\nDISTRIBUIÇÃO DE SINAIS:")
signal_counts = data_test["Signal"].value_counts()
for signal, count in signal_counts.items():
    print(f"{signal}: {count} ({count/len(data_test)*100:.1f}%)")